In [1]:
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D,Flatten,Dense
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile
import glob
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
CLASS_NAMES = ['2C0', 'C0','C0C2']

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(r'D:\\important_for_study_master\\DoAn\\New folder\\data\\data_using_opencv\\train',
                                                 target_size = (100, 100),
                                                 batch_size = 32,
                                                 classes  = CLASS_NAMES ,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(r'D:\\important_for_study_master\\DoAn\\New folder\\data\\data_using_opencv\\test',
                                            target_size = (100, 100),
                                            batch_size = 32,
                                            classes  = CLASS_NAMES ,
                                            class_mode = 'categorical')

Found 604 images belonging to 3 classes.
Found 143 images belonging to 3 classes.


In [59]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (11,11), activation='relu',input_shape=(100, 100, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(3, 3),
    tf.keras.layers.Dropout(0.25),
    
    tf.keras.layers.Conv2D(32, (5,5), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(3,3),
    tf.keras.layers.Dropout(0.25),
      
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [60]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 90, 90, 16)        5824      
                                                                 
 batch_normalization_20 (Ba  (None, 90, 90, 16)        64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 30, 30, 16)        0         
 ng2D)                                                           
                                                                 
 dropout_20 (Dropout)        (None, 30, 30, 16)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 26, 26, 32)        12832     
                                                                 
 batch_normalization_21 (Ba  (None, 26, 26, 32)       

In [61]:
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
                            optimizer=opt,
                            metrics=['accuracy'])

In [62]:
history = model.fit(training_set,epochs = 100)

Epoch 1/100
19/19 [==============================] - 4s 165ms/step - loss: 1.2123 - accuracy: 0.6407
Epoch 2/100
19/19 [==============================] - 3s 164ms/step - loss: 0.4608 - accuracy: 0.8096
Epoch 3/100
19/19 [==============================] - 3s 162ms/step - loss: 0.5566 - accuracy: 0.8079
Epoch 4/100
19/19 [==============================] - 3s 165ms/step - loss: 0.4220 - accuracy: 0.8576
Epoch 5/100
19/19 [==============================] - 3s 163ms/step - loss: 0.2427 - accuracy: 0.8891
Epoch 6/100
19/19 [==============================] - 3s 164ms/step - loss: 0.2456 - accuracy: 0.9123
Epoch 7/100
19/19 [==============================] - 3s 167ms/step - loss: 0.2503 - accuracy: 0.9139
Epoch 8/100
19/19 [==============================] - 3s 167ms/step - loss: 0.1835 - accuracy: 0.9305
Epoch 9/100
19/19 [==============================] - 3s 164ms/step - loss: 0.3642 - accuracy: 0.8907
Epoch 10/100
19/19 [==============================] - 3s 163ms/step - loss: 0.1832 - accura

In [63]:
test_loss, test_accuracy = model.evaluate(test_set, verbose=2)
print("Loss  : ", test_loss)
print("Accuracy  :",test_accuracy)

5/5 - 0s - loss: 0.5742 - accuracy: 0.8741 - 287ms/epoch - 57ms/step
Loss  :  0.574154257774353
Accuracy  : 0.8741258978843689


In [64]:
model.save('CNNv1.h5')

In [65]:
savedModel=load_model('CNNv1.h5')
savedModel.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 90, 90, 16)        5824      
                                                                 
 batch_normalization_20 (Ba  (None, 90, 90, 16)        64        
 tchNormalization)                                               
                                                                 
 max_pooling2d_20 (MaxPooli  (None, 30, 30, 16)        0         
 ng2D)                                                           
                                                                 
 dropout_20 (Dropout)        (None, 30, 30, 16)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 26, 26, 32)        12832     
                                                                 
 batch_normalization_21 (Ba  (None, 26, 26, 32)       

In [69]:
test_path = "D:\\important_for_study_master\\DoAn\\New folder\\data\\test\\Test_data\\C0C2\\"
total = 0
rs = 0
for filename in glob.glob(test_path+"*"):
    print(filename)
    test_image = image.load_img(filename, target_size = (100,100))
    test_image = np.expand_dims(test_image, axis=0)
    result = savedModel.predict(test_image)
    
    if result[0][0] == 1:
        print("2C0")
        
    if result[0][1] == 1:
        print("C0")
        
    elif result[0][2] == 1:
         print("C0C2")
         rs+=1
    
    total+=1
print('Nong do C0 dung duoc "%" la: '+str(rs/total))

D:\important_for_study_master\DoAn\New folder\data\test\Test_data\C0C2\0.jpg
1/1 [==============================] - 0s 18ms/step
2C0
D:\important_for_study_master\DoAn\New folder\data\test\Test_data\C0C2\1.jpg
1/1 [==============================] - 0s 17ms/step
C0C2
D:\important_for_study_master\DoAn\New folder\data\test\Test_data\C0C2\10.jpg
1/1 [==============================] - 0s 17ms/step
2C0
D:\important_for_study_master\DoAn\New folder\data\test\Test_data\C0C2\100.jpg
1/1 [==============================] - 0s 18ms/step
2C0
D:\important_for_study_master\DoAn\New folder\data\test\Test_data\C0C2\101.jpg
1/1 [==============================] - 0s 17ms/step
2C0
D:\important_for_study_master\DoAn\New folder\data\test\Test_data\C0C2\102.jpg
1/1 [==============================] - 0s 17ms/step
2C0
D:\important_for_study_master\DoAn\New folder\data\test\Test_data\C0C2\103.jpg
1/1 [==============================] - 0s 18ms/step
2C0
D:\important_for_study_master\DoAn\New folder\data\test\Tes